### functionality

### required directory structure

### imports and inputs

In [67]:
import numpy as np
import pandas as pd
import seaborn as sb
import math
import os

from packages import miscellaneous as mc
from packages import timeconvert as tc
from packages import plotstyle as ps

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
sb.set_theme(style="white",font="serif")

date = '8_9_2021 (evening)'

ymax, ymin = 2.5,-2.5

timeRange = ["18","24"] # Start and end hour in 24-hour time: ["HH","HH"]. For title only.

# Code is general to antennas, but plot arrangements are specific, so structure of lists below is fixed.
refants = ['DA59','DV11','DA49','DV12','PM01','DV07'] # Must be three pairs [A,A,B,B,C,C], where A,A are central.
pairLoc = ['Center','Southwest','East'] # Must be three corresponding locations [A,B,C].

directory = '/Users/colemeyer/Documents/ALMA Internship/Observational Data/' # Directory of "Observational Data", assuming specified structure

### general initializations

In [68]:
weaants = ['Meteo129','Meteo130','Meteo131','Meteo201','Meteo309','Meteo410','MeteoCentral']

directory = directory+'/'+date
genDir = directory+'/asdm/'

numObs = mc.numObs(genDir)
    
# Create new "offsetplots" directory within "Observational Data" subdirectory
if not os.path.isdir(directory+'/phaseplots/'):
    os.mkdir(directory+'/phaseplots/')

# magnitude w/ mean-based normalization

### specific initializations

In [69]:
magPhaseProbs = np.empty((np.shape(refants)[0],numObs),dtype='U25')
magPhase = np.empty((np.shape(refants)[0],numObs),dtype='float')
x = range(1,numObs+1)
xticks = range(numObs+1)

########## magnitude-specific phase data

# Data extraction from offsetdata directory
for ant in range(np.shape(refants)[0]):
    arr = pd.read_csv(directory+'/offsetdata/'+refants[ant]+'.csv')
    arr = arr.loc[:,'xPhaseAvg']
    for obs in range(numObs):
        # If there's a problem in the data, prep it for the next loop
        if np.isnan(arr.iloc[obs]):
            L = np.where(magPhaseProbs[ant,:] == "")[0][0]
            magPhaseProbs[ant,L] = obs
            magPhase[ant,obs] = np.nan
        # Else, transfer the data
        else: magPhase[ant,obs] = arr.iloc[obs]
            
# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update magPhaseProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(refants)[0]):
    if magPhaseProbs[ant,0] != "":
        for prob in range(np.where(magPhaseProbs[ant,:] == "")[0][0]):
            if int(magPhaseProbs[ant,prob]) == 0: magPhase[ant,int(magPhaseProbs[ant,prob])] = magPhase[ant,int(magPhaseProbs[ant,prob])+1]
            else: magPhase[ant,int(magPhaseProbs[ant,prob])] = magPhase[ant,int(magPhaseProbs[ant,prob])-1]
        
# Prepare time labels
timeSeries = []
temp = pd.read_csv(directory+'/offsetdata/'+refants[0]+'.csv')[['time']]
for time in range(temp.shape[0]):
    timeSeries.append(tc.UnixtoFormattedTime(temp.loc[time]))
timeSeries.append("")

### mean-based normalization

In [70]:
for ant in range(np.shape(refants)[0]):
    offset = np.average(magPhase[ant,:])
    magPhase[ant,:] = magPhase[ant,:]-offset

### combined plot initializations

In [71]:
magPhaseProbsAvg = np.empty((int(np.shape(refants)[0]/2),numObs),dtype='U25')
magPhaseAvg = np.zeros((int(np.shape(refants)[0]/2),numObs),dtype='float')

for ant in range(int(np.shape(refants)[0]/2)):
    magPhaseAvg[ant,:] = (magPhase[ant*2,:] + magPhase[ant*2+1,:]) / 2
        
for ant in range(np.shape(refants)[0]):
    pair = ant // 2
    if magPhaseProbs[ant,0] != "":
        for prob in range(np.where(magPhaseProbs[ant,:] == "")[0][0]):
            L = np.where(magPhaseProbsAvg[pair,:] == "")[0][0]
            magPhaseProbsAvg[pair,L] = magPhaseProbs[ant,prob]
for pair in range(int(np.shape(refants)[0]/2)):
    if magPhaseProbsAvg[pair,0] != "":
        for prob in range(0):
            if int(magPhaseProbsAvg[pair,prob]) == 0: magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob]),:] = magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob])+1]
            else: magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob]),:] = magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob])-1]

                
for pair in range(int(np.shape(refants)[0]/2)):
    offset = np.average(magPhaseAvg[pair,:])
    magPhaseAvg[pair,:] = magPhaseAvg[pair,:] - offset

### extrema

In [72]:
extrema = np.zeros((np.shape(refants)[0],2),dtype='float')
for ant in range(np.shape(refants)[0]):
    extrema[ant,0], extrema[ant,1] = np.max(magPhase[ant,:]), np.min(magPhase[ant,:])
    
temp = np.max([abs(np.max(extrema[:,0])), abs(np.min(extrema[:,1]))])
ymax, ymin = ymax, ymin

## plot 

In [73]:
fig, axes = plt.subplots(2,2,figsize=(10,10),sharey=True,sharex=True)

ax = [axes[0,0],axes[0,1],axes[1,0],axes[1,1]]

for i in range(3):
    temp = pd.DataFrame(np.column_stack((x,magPhase[i*2,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='r')
    temp = pd.DataFrame(np.column_stack((x,magPhase[i*2+1,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='b')
    
colors = ['r','b','g']
for i in range(3):
    temp = pd.DataFrame(np.column_stack((x,magPhaseAvg[i,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[3], x='x', y='y', data=temp, color=colors[i])
    
for ant in range(len(refants)):
    if magPhaseProbs[ant,0] != "":
        for row in range(np.where(magPhaseProbs[ant,:] == "")[0][0]):
            ec = colors[ant%2]
            ax[ant//2].scatter(float(magPhaseProbs[ant,row])+1,float(magPhase[ant,int(magPhaseProbs[ant,row])]),s=500,c='none',edgecolor=ec,linewidth=2)
            
    if magPhaseProbsAvg[ant//2,0] != "":
        for row in range(np.where(magPhaseProbsAvg[ant//2,:] == "")[0][0]):
            ec = colors[ant//2]
            ax[3].scatter(float(magPhaseProbsAvg[ant//2,row])+1,float(magPhaseAvg[ant//2,int(magPhaseProbsAvg[ant//2,row])]),s=500,c='none',edgecolor=ec,linewidth=2)

handles = []
for i in range(3):
    handles.append([Line2D([0],[0],marker='o',color='r',label=refants[i*2],linewidth=0),
          Line2D([0],[0],marker='o',color='b',label=refants[i*2+1],linewidth=0)])
handles.append([Line2D([0],[0],marker='o',color='r',label=pairLoc[0]+' Pair Avg',linewidth=0),
          Line2D([0],[0],marker='o',color='b',label=pairLoc[1]+' Pair Avg',linewidth=0),
          Line2D([0],[0],marker='o',color='g',label=pairLoc[2]+' Pair Avg',linewidth=0)])
            
names = [pairLoc[0]+' Pair, ',pairLoc[1]+' Pair, ',pairLoc[2]+' Pair, ','Combined, ']
for i in range(4):
    ps.plotNonDescript(ax[i],title='x-Phase Average, '+names[i]+str(numObs)+'-Obs. Period,\n'+
                   timeRange[0]+':00 - '+timeRange[1]+':00, Mean-Based Normalization',ymax=ymax,
                   ymin=ymin,titlesize=11,xlabel='Observation (not evenly spaced)',ylabel='x-Phase Average',
                       xticks=xticks,xticklabels=timeSeries,yticks=True,handles=handles[i])

plt.tight_layout()

#plt.show()
plt.savefig(directory+'/phaseplots/magPhase.png', bbox_inches='tight')
plt.close()

# delta w/ mean-based normalization

### specific initializations

In [74]:
deltaPhaseProbs = np.empty((np.shape(refants)[0],numObs-1),dtype='U25')
deltaPhase = np.empty((np.shape(refants)[0],numObs-1),dtype='float')
x = range(1,numObs)
xticks = np.array(range(numObs+1),dtype='float')-0.5

########## deltaPhase-specific pos data

# Data extraction from offsetdata directory
for ant in range(np.shape(refants)[0]):
    arr = pd.read_csv(directory+'/offsetdata/'+refants[ant]+'.csv')
    arr = arr.loc[:,'xPhaseAvg']

    for obs in range(numObs-1):
        # If there's a problem in the data, prep it for the next loop
        if np.isnan(arr.iloc[obs+1]) or np.isnan(arr.iloc[obs]):
            L = np.where(deltaPhaseProbs[ant,:] == "")[0][0]
            deltaPhaseProbs[ant,L] = obs
            deltaPhase[ant,obs] = np.nan
        # Else, transfer the data
        else: deltaPhase[ant,obs] = arr.iloc[obs+1]-arr.iloc[obs]

# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update deltaPhaseProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(refants)[0]):
    if deltaPhaseProbs[ant,0] != "":
        for prob in range(np.where(deltaPhaseProbs[ant,:] == "")[0][0]):
            if int(deltaPhaseProbs[ant,prob]) == 0: deltaPhase[ant,int(deltaPhaseProbs[ant,prob])] = deltaPhase[ant,int(deltaPhaseProbs[ant,prob])+1]
            else: deltaPhase[ant,int(deltaPhaseProbs[ant,prob])] = deltaPhase[ant,int(deltaPhaseProbs[ant,prob])-1]

# Prepare time labels
timeSeries = []
temp = pd.read_csv(directory+'/offsetdata/'+refants[0]+'.csv')[['time']]
for time in range(temp.shape[0]):
    timeSeries.append(tc.UnixtoFormattedTime(temp.loc[time]))
    
timeSeries.append("")

### mean-based normalization

In [75]:
for ant in range(np.shape(refants)[0]):
    for coord in range(3):
        offset = np.average(deltaPhase[ant,:])
        deltaPhase[ant,:] = deltaPhase[ant,:]-offset

### combined plot initializations

In [76]:
deltaPhaseProbsAvg = np.empty((int(np.shape(refants)[0]/2),numObs-1),dtype='U25')
deltaPhaseAvg = np.zeros((int(np.shape(refants)[0]/2),numObs-1),dtype='float')

for ant in range(int(np.shape(refants)[0]/2)):
    deltaPhaseAvg[ant,:] = (deltaPhase[ant*2,:] + deltaPhase[ant*2+1,:]) / 2
        
for ant in range(np.shape(refants)[0]):
    pair = ant // 2
    if deltaPhaseProbs[ant,0] != "":
        for prob in range(np.where(deltaPhaseProbs[ant,:] == "")[0][0]):
            L = np.where(deltaPhaseProbsAvg[pair,:] == "")[0][0]
            deltaPhaseProbsAvg[pair,L] = deltaPhaseProbs[ant,prob]
for pair in range(int(np.shape(refants)[0]/2)):
    if deltaPhaseProbsAvg[pair,0] != "":
        for prob in range(0):
            if int(deltaPhaseProbsAvg[pair,prob]) == 0: deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])] = deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])+1]
            else: deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])] = deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])-1]

                
for pair in range(int(np.shape(refants)[0]/2)):
    for coord in range(3):
        offset = np.average(deltaPhaseAvg[pair,:])
        deltaPhaseAvg[pair,:] = deltaPhaseAvg[pair,:] - offset

### extrema

In [77]:
extrema = np.zeros((np.shape(refants)[0],2),dtype='float')
for ant in range(np.shape(refants)[0]):
    extrema[ant,0], extrema[ant,1] = np.max(deltaPhase[ant,:]), np.min(deltaPhase[ant,:])
    
temp = np.max([abs(np.max(extrema[:,0])), abs(np.min(extrema[:,1]))])
ymax, ymin = ymax, ymin

## plot

In [78]:
fig, axes = plt.subplots(2,2,figsize=(10,10),sharey=True,sharex=True)

ax = [axes[0,0],axes[0,1],axes[1,0],axes[1,1]]

for i in range(3):
    temp = pd.DataFrame(np.column_stack((x,deltaPhase[i*2,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='r')
    temp = pd.DataFrame(np.column_stack((x,deltaPhase[i*2+1,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='b')
    
colors = ['r','b','g']
for i in range(3):
    temp = pd.DataFrame(np.column_stack((x,deltaPhaseAvg[i,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[3], x='x', y='y', data=temp, color=colors[i])
    
for ant in range(len(refants)):
    if deltaPhaseProbs[ant,0] != "":
        for row in range(np.where(deltaPhaseProbs[ant,:] == "")[0][0]):
            ec = colors[ant%2]
            ax[ant//2].scatter(float(deltaPhaseProbs[ant,row])+1,float(deltaPhase[ant,int(deltaPhaseProbs[ant,row])]),s=500,c='none',edgecolor=ec,linewidth=2)
            
    if deltaPhaseProbsAvg[ant//2,0] != "":
        for row in range(np.where(deltaPhaseProbsAvg[ant//2,:] == "")[0][0]):
            ec = colors[ant//2]
            ax[3].scatter(float(deltaPhaseProbsAvg[ant//2,row])+1,float(deltaPhaseAvg[ant//2,int(deltaPhaseProbsAvg[ant//2,row])]),s=500,c='none',edgecolor=ec,linewidth=2)

handles = []
for i in range(3):
    handles.append([Line2D([0],[0],marker='o',color='r',label=refants[i*2],linewidth=0),
          Line2D([0],[0],marker='o',color='b',label=refants[i*2+1],linewidth=0)])
handles.append([Line2D([0],[0],marker='o',color='r',label=pairLoc[0]+' Pair Avg',linewidth=0),
          Line2D([0],[0],marker='o',color='b',label=pairLoc[1]+' Pair Avg',linewidth=0),
          Line2D([0],[0],marker='o',color='g',label=pairLoc[2]+' Pair Avg',linewidth=0)])
            
names = [pairLoc[0]+' Pair, ',pairLoc[1]+' Pair, ',pairLoc[2]+' Pair, ','Combined, ']
for i in range(4):
    ps.plotNonDescript(ax[i],title='x-Phase ΔAverage, '+names[i]+str(numObs)+'-Obs. Period,\n'+
                   timeRange[0]+':00 - '+timeRange[1]+':00, Mean-Based Normalization',ymax=ymax,
                   ymin=ymin,titlesize=11,xlabel='Observation (not evenly spaced)',ylabel='x-Phase ΔAverage',
                       xticks=xticks,xticklabels=timeSeries,yticks=True,handles=handles[i])

plt.tight_layout()

#plt.show()
plt.savefig(directory+'/phaseplots/deltaPhase.png', bbox_inches='tight')
plt.close()

# mag (offset & weather) w/ mean-based normalization

### specific initializations

In [79]:
magWeaProbs = np.empty((np.shape(weaants)[0],numObs),dtype='U25')
magWea = np.empty((np.shape(weaants)[0],numObs,6),dtype='float')
x = range(1,numObs+1) # 1,numObs+1
xticks = range(numObs+1)

########## magnitude-specific wea data

# Data extraction from weatherdata directory
for ant in range(np.shape(weaants)[0]):
    arr = pd.read_csv(directory+'/weatherdata/'+weaants[ant]+'.csv')
    arr = arr.loc[:,'pressure':'windMax']
    for obs in range(numObs):
        # If there's a problem in the data, prep it for the next loop
        if arr.iloc[obs].isnull().values.any():
            L = np.where(magWeaProbs[ant,:] == "")[0][0]
            magWeaProbs[ant,L] = obs
            magWea[ant,obs,:] = np.nan
        # Else, transfer the data
        else: magWea[ant,obs,:] = arr.iloc[obs]
            
# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update magProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(weaants)[0]):
    if magWeaProbs[ant,0] != "":
        for prob in range(np.where(magWeaProbs[ant,:] == "")[0][0]):
            if int(magWeaProbs[ant,prob]) == 0: magWea[ant,int(magWeaProbs[ant,prob]),:] = magWea[ant,int(magWeaProbs[ant,prob])+1,:]
            else: magWea[ant,int(magWeaProbs[ant,prob]),:] = magWea[ant,int(magWeaProbs[ant,prob])-1,:]
                
# Remove windDir and windMax
temp = np.empty((np.shape(weaants)[0],numObs,4))
temp[:,:,0] = magWea[:,:,0]
temp[:,:,1] = magWea[:,:,1]
temp[:,:,2] = magWea[:,:,2]
temp[:,:,3] = magWea[:,:,4]
magWea = temp

### mean-based normalization

In [80]:
for ant in range(np.shape(weaants)[0]):
    for i in range(4):
        offset = np.average(magWea[ant,:,i])
        magWea[ant,:,i] = magWea[ant,:,i]-offset

### mean-based scaling

In [81]:
posPoints = []
negPoints = []
for i in range(numObs):
    if magPhaseAvg[0,i] > 0: posPoints.append(magPhaseAvg[0,i])     
avg1 = np.average(np.array(posPoints,dtype='float'))

if os.listdir(genDir)[0][:3] == "uid": uid = os.listdir(genDir)[0]
else: uid = os.listdir(genDir)[1]

counter = 1
antMissing = True    
while antMissing == True:
    antMissing = False
    try:
        uid = os.listdir(genDir)[1+counter]
        nearestWeatherStation = mc.nearestWeatherStation(genDir+uid+'/',[refants[0],refants[1]])
    except:
        antMissing = True
        
    if antMissing == False: nearestWeatherStation = mc.nearestWeatherStation(genDir+uid+'/',[refants[0],refants[1]])
    counter += 1    
    
for ant in range(len(weaants)):
    if nearestWeatherStation == weaants[ant]:
        nearestWeatherStation = ant

avg2 = np.empty((4),dtype='float')
for i in range(4):
    posPoints = []
    negPoints = []
    for j in range(numObs):
        if magWea[nearestWeatherStation,j,i] > 0: posPoints.append(magWea[nearestWeatherStation,j,i])
    avg2[i] = np.average(np.array(posPoints,dtype='float'))

scaleFactor = avg1 / avg2

for i in range(4): magWea[nearestWeatherStation,:,i] = magWea[nearestWeatherStation,:,i] * scaleFactor[i]

### extrema

In [82]:
extrema = np.zeros((np.shape(refants)[0],2),dtype='float')
for ant in range(2):
    extrema[ant,0], extrema[ant,1] = np.max(magPhase[ant,:]), np.min(magPhase[ant,:])
    
extremaWea = np.zeros((4,2),dtype='float')
for i in range(4):
    extremaWea[i,0], extremaWea[i,1] = np.max(magWea[nearestWeatherStation,:,i]), np.min(magWea[nearestWeatherStation,:,i])
    
allExtrema = [extrema[0,0], extrema[0,1], extrema[1,0], extrema[1,1]]
for i in range(4):
    allExtrema.append(extremaWea[i,0])
    allExtrema.append(extremaWea[i,1])
allExtrema = [abs(element) for element in allExtrema]
    
temp = np.max(allExtrema)
ymax, ymin = ymax, ymin

## plot

In [83]:
fig, axes = plt.subplots(2,2,figsize=(10,10),sharey=True,sharex=True)

ax = [axes[0,0],axes[0,1],axes[1,0],axes[1,1]]
weaLabels = ['Pressure','Rel. Humidity','Temperature','WindSpeed']

handles = []
for i in range(4):
    temp = pd.DataFrame(np.column_stack((x,magPhaseAvg[0,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='r')
    
    temp = pd.DataFrame(np.column_stack((x,magWea[nearestWeatherStation,:,i])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='b')
    
    handles.append([Line2D([0],[0],marker='o',color='r',label=pairLoc[0]+' Pair Avg',linewidth=0),
          Line2D([0],[0],marker='o',color='b',label=weaLabels[i],linewidth=0)])
    
    if magPhaseProbsAvg[0,0] != "":
        for row in range(np.where(magPhaseProbsAvg[0,:] == "")[0][0]):
            ax[i].scatter(float(magPhaseProbsAvg[0,row])+1,float(magPhaseAvg[0,int(magPhaseProbsAvg[0,row])]),s=500,c='none',edgecolor='r',linewidth=2)
            
    if magWeaProbs[0,0] != "":
        for row in range(np.where(magWeaProbs[0,:] == "")[0][0]):
            ax[i].scatter(float(magWeaProbs[0,row])+1,float(magWea[0,int(magWeaProbs[0,row]),index]),s=500,c='none',edgecolor='b',linewidth=2)

for i in range(4):
    ps.plotNonDescript(ax[i],title=weaLabels[i]+' vs. x-Phase Average, '+str(numObs)+'-Obs. Period,\n Mean-Based Normalization and Scaling',ymax=ymax,
                   ymin=ymin,titlesize=11,xlabel='Observation (not evenly spaced)',ylabel='x-Phase Average and Weather',
                       xticks=xticks,xticklabels=timeSeries,yticks=True,handles=handles[i])

plt.tight_layout()

#plt.show()
plt.savefig(directory+'/phaseplots/magPhaseWea.png', bbox_inches='tight')
plt.close()

# delta (offset & weather) w/ mean-based normalization

### specific initializations

In [84]:
deltaWeaProbs = np.empty((np.shape(weaants)[0],numObs-1),dtype='U25')
deltaWea = np.empty((np.shape(weaants)[0],numObs-1,6),dtype='float')
x = range(1,numObs)
xticks = np.array(range(numObs+1),dtype='float')-0.5

########## delta-specific wea data

# Data extraction from weatherdata directory
for ant in range(np.shape(weaants)[0]):
    arr = pd.read_csv(directory+'/weatherdata/'+weaants[ant]+'.csv')
    arr = arr.loc[:,'pressure':'windMax']
    for obs in range(numObs-1):
        # If there's a problem in the data, prep it for the next loop
        if arr.iloc[obs+1].isnull().values.any() or arr.iloc[obs].isnull().values.any():
            L = np.where(deltaWeaProbs[ant,:] == "")[0][0]
            deltaWeaProbs[ant,L] = obs
            deltaWea[ant,obs,:] = np.nan
        # Else, transfer the data
        else: deltaWea[ant,obs,:] = arr.iloc[obs+1]-arr.iloc[obs]
            
# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update magProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(weaants)[0]):
    if deltaWeaProbs[ant,0] != "":
        for prob in range(np.where(deltaWeaProbs[ant,:] == "")[0][0]):
            if int(deltaWeaProbs[ant,prob]) == 0: deltaWea[ant,int(deltaWeaProbs[ant,prob]),:] = deltaWea[ant,int(deltaWeaProbs[ant,prob])+1,:]
            else: deltaWea[ant,int(deltaWeaProbs[ant,prob]),:] = deltaWea[ant,int(deltaWeaProbs[ant,prob])-1,:]
                
# Remove windDir and windMax
temp = np.empty((np.shape(weaants)[0],numObs-1,4))
temp[:,:,0] = deltaWea[:,:,0]
temp[:,:,1] = deltaWea[:,:,1]
temp[:,:,2] = deltaWea[:,:,2]
temp[:,:,3] = deltaWea[:,:,4]
deltaWea = temp

### mean-based normalization

In [85]:
for ant in range(np.shape(weaants)[0]):
    for i in range(4):
        offset = np.average(deltaWea[ant,:,i])
        deltaWea[ant,:,i] = deltaWea[ant,:,i]-offset

### mean-based scaling

In [86]:
posPoints = []
negPoints = []
for i in range(numObs-1):
    if deltaPhaseAvg[0,i] > 0: posPoints.append(deltaPhaseAvg[0,i])     
avg1 = np.average(np.array(posPoints,dtype='float'))

if os.listdir(genDir)[0][:3] == "uid": uid = os.listdir(genDir)[0]
else: uid = os.listdir(genDir)[1]

counter = 1
antMissing = True    
while antMissing == True:
    antMissing = False
    try:
        uid = os.listdir(genDir)[1+counter]
        nearestWeatherStation = mc.nearestWeatherStation(genDir+uid+'/',[refants[0],refants[1]])
    except:
        antMissing = True
        
    if antMissing == False: nearestWeatherStation = mc.nearestWeatherStation(genDir+uid+'/',[refants[0],refants[1]])
    counter += 1

for ant in range(len(weaants)):
    if nearestWeatherStation == weaants[ant]:
        nearestWeatherStation = ant

avg2 = np.empty((4),dtype='float')
for i in range(4):
    posPoints = []
    negPoints = []
    for j in range(numObs-1):
        if deltaWea[nearestWeatherStation,j,i] > 0: posPoints.append(deltaWea[nearestWeatherStation,j,i])
    avg2[i] = np.average(np.array(posPoints,dtype='float'))

scaleFactor = avg1 / avg2

for i in range(4): deltaWea[nearestWeatherStation,:,i] = deltaWea[nearestWeatherStation,:,i] * scaleFactor[i]

### extrema

In [87]:
extrema = np.zeros((np.shape(refants)[0],2),dtype='float')
for ant in range(2):
    extrema[ant,0], extrema[ant,1] = np.max(deltaPhase[ant,:]), np.min(deltaPhase[ant,:])
    
extremaWea = np.zeros((4,2),dtype='float')
for i in range(4):
    extremaWea[i,0], extremaWea[i,1] = np.max(deltaWea[nearestWeatherStation,:,i]), np.min(deltaWea[nearestWeatherStation,:,i])
    
allExtrema = [extrema[0,0], extrema[0,1], extrema[1,0], extrema[1,1]]
for i in range(4):
    allExtrema.append(extremaWea[i,0])
    allExtrema.append(extremaWea[i,1])
allExtrema = [abs(element) for element in allExtrema]
    
temp = np.max(allExtrema)
ymax, ymin = ymax, ymin

## plot

In [88]:
fig, axes = plt.subplots(2,2,figsize=(10,10),sharey=True,sharex=True)

ax = [axes[0,0],axes[0,1],axes[1,0],axes[1,1]]
weaLabels = ['Pressure','Rel. Humidity','Temperature','WindSpeed']

handles = []
for i in range(4):
    temp = pd.DataFrame(np.column_stack((x,deltaPhaseAvg[0,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='r')
    
    temp = pd.DataFrame(np.column_stack((x,deltaWea[nearestWeatherStation,:,i])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='b')
    
    handles.append([Line2D([0],[0],marker='o',color='r',label=pairLoc[0]+' Pair Avg',linewidth=0),
          Line2D([0],[0],marker='o',color='b',label=weaLabels[i],linewidth=0)])
    
    if deltaPhaseProbsAvg[0,0] != "":
        for row in range(np.where(deltaPhaseProbsAvg[0,:] == "")[0][0]):
            ax[i].scatter(float(deltaPhaseProbsAvg[0,row])+1,float(deltaPhaseAvg[0,int(deltaPhaseProbsAvg[0,row])]),s=500,c='none',edgecolor='r',linewidth=2)

    if deltaWeaProbs[0,0] != "":
        for row in range(np.where(deltaWeaProbs[0,:] == "")[0][0]):
            ax[i].scatter(float(deltaWeaProbs[0,row])+1,float(deltaWea[0,int(deltaWeaProbs[0,row]),index]),s=500,c='none',edgecolor='b',linewidth=2)
            
for i in range(4):
    ps.plotNonDescript(ax[i],title='Δ'+weaLabels[i]+' vs. x-Phase ΔAverage, '+str(numObs)+'-Obs. Period,\n Mean-Based Normalization and Scaling',ymax=ymax,
                   ymin=ymin,titlesize=11,xlabel='Observation (not evenly spaced)',ylabel='x-Phase ΔAverage and ΔWeather',
                       xticks=xticks,xticklabels=timeSeries,yticks=True,handles=handles[i])

plt.tight_layout()

#plt.show()
plt.savefig(directory+'/phaseplots/deltaPhaseWea.png', bbox_inches='tight')
plt.close()

print(date)

8_9_2021 (evening)
